# Preprations

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_path = "/content/drive/MyDrive/ML_VIsion/model_files/"

In [3]:
!git clone https://github.com/SirAfshin/ML_Graduate_Project_TA.git

Cloning into 'ML_Graduate_Project_TA'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 42 (delta 12), reused 25 (delta 8), pack-reused 0
Unpacking objects: 100% (42/42), 11.84 KiB | 1.32 MiB/s, done.


In [4]:
# run this cell twice for it to work !!!!!!!!!!!!
import warnings
warnings.filterwarnings('ignore')
!pip install -q git+https://github.com/RobustBench/robustbench.git@2d630bc9e8d1cf50d46a4dda65fd36850e3ef769

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.3.5 which is incompatible.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.25.1 which is incompatible.
tweepy 4.13.0 requires requests<3,>=2.27.0, but you have requests 2.25

In [5]:
# because twice -> used 2 identical cells
# run this cell twice for it to work !!!!!!!!!!!!
import warnings
warnings.filterwarnings('ignore')
!pip install -q git+https://github.com/RobustBench/robustbench.git@2d630bc9e8d1cf50d46a4dda65fd36850e3ef769

  Preparing metadata (setup.py) ... done


In [6]:
!cp /content/ML_Graduate_Project_TA/Phase_1/utils.py /content/

In [7]:
!mkdir /content/models

In [8]:
model_path = drive_path+"*"
!cp {model_path} /content/models

In [9]:
!pip install git+https://github.com/jeromerony/fast_adversarial

  Cloning https://github.com/jeromerony/fast_adversarial to /tmp/pip-req-build-rv_nr2dj
  Running command git clone --filter=blob:none --quiet https://github.com/jeromerony/fast_adversarial /tmp/pip-req-build-rv_nr2dj
  Resolved https://github.com/jeromerony/fast_adversarial to commit 45210b7c79e2deaeac9845d6c901dc2580d6e316
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
  Created wheel for fast-adversarial: filename=fast_adversarial-0.1-py3-none-any.whl size=29326 sha256=d85b87d8d8c23ade11016e7bebddac51537c11fb04d1480a3699fd893b51cf63
  Stored in directory: /tmp/pip-ephem-wheel-cache-d2o3g0jx/wheels/d6/68/b8/b4ae3dd5ec7d260836ef6dbd54028

# Phase3 - DDN

In [10]:
import argparse
import torch
import time
from torch.utils import data
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid

from fast_adv.models.mnist import SmallCNN
from fast_adv.attacks import DDN, CarliniWagnerL2
from fast_adv.utils import requires_grad_, l2_norm
import matplotlib.pyplot as plt
import os
%matplotlib inline

# import architecture of models
from models.resnet_addpool import ResNet50 as AddPool50
from models.resnet_nobatch import ResNet50 as NoBatch5
from models.resnet_nopool import ResNet50 as NoPool50
from models.resnet_standard import ResNet50 as STD50


from robustbench import load_cifar10, load_model
import torch.nn as nn
from utils import *


In [11]:
# set random seed
torch.manual_seed(67) #42
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# path of all model saved files
model_paths = {'std': '/content/models/resnet_standard.pth',\
               'nopool': '/content/models/resnet_nopool.pth',\
               'addpool': '/content/models/resnet_addpool.pth',\
               'nobatch': '/content/models/resnet_nobatch.pth'}

In [12]:
# # Loading the data
# dataset = datasets.CIFAR10('/content/CIFAR10/',train=False,transform=transforms.ToTensor(),download=True)
# loader = data.DataLoader(dataset, shuffle=False, batch_size=100)

# x, y = next(iter(loader))
# x = x.to(device)
# y = y.to(device)

# plt.imshow(make_grid(x.to('cpu'), nrow=16).permute(1,2,0))
# plt.axis('off');

In [13]:
# loading dataset --> CIFAR10

batch_size = 200
print('Loading data')
dataset_test = datasets.CIFAR10('/content/CIFAR10', train=False,transform=transforms.ToTensor(),download=True)
loader_test = data.DataLoader(dataset_test, shuffle=False, batch_size= batch_size)

x, y = next(iter(loader_test))
x = x.to(device)
y = y.to(device)

# Inverse normalization
mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(1, 3, 1, 1)
std = torch.tensor([0.2023, 0.1994, 0.2010]).view(1, 3, 1, 1)

# Move std and mean tensors to the same device as x
std = std.to(x.device)
mean = mean.to(x.device)

Loading data


100%|██████████| 170498071/170498071 [00:01<00:00, 99249239.13it/s] 


Extracting /content/CIFAR10/cifar-10-python.tar.gz to /content/CIFAR10


In [14]:
from logging import raiseExceptions
print('Check models availability:')

if not os.path.exists(model_paths['std']):
  raise Exception("Sorry, no model std available")

if not os.path.exists(model_paths['nopool']):
  raise Exception("Sorry, no model std available")

if not os.path.exists(model_paths['addpool']):
  raise Exception("Sorry, no model std available")

if not os.path.exists(model_paths['nobatch']):
  raise Exception("Sorry, no model std available")

print('... All models are available')

Check models availability:
... All models are available


In [15]:
# load model
def load_resnet50(model_class,model_path,mean,std):
  print('Loading model')
  model = model_class()
  model = nn.DataParallel(model)
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['net'])
  model = NormalizedModel(model, mean =  mean , std = std)
  model.eval().to(device)
  requires_grad_(model, False)
  return model

In [16]:
def run_ddn(model,x,y):
  print('Running DDN attack')
  attacker = DDN(steps=1000, device=device)
  start = time.time()
  ddn_atk = attacker.attack(model, x, labels=y, targeted=False)
  ddn_time = time.time() - start

  # Save images
  all_imgs = torch.cat((x, ddn_atk))
  save_image(all_imgs, 'images_and_attacks.png', nrow=16, pad_value=0)

  pred_ddn = model(ddn_atk).argmax(dim=1).cpu()
  print(5*"*"+"DDN-1000"+5*"*")
  print("Duration {:.1f}s \nSuccess: {:.2f}% \nMean L2: {:.4f} \nMedian L2: {:.4f}".format(\
      ddn_time,(pred_ddn != y.cpu()).float().mean().item() * 100,l2_norm(ddn_atk - x).mean().item(),l2_norm(ddn_atk - x).median().item()))


In [17]:
def eval_ddn(model_class,model_path,x,y,mean,std):
  # Empty cuda cache
  torch.cuda.empty_cache()
  # load model
  model = load_resnet50(model_class,model_path,mean,std)
  run_ddn(model,x,y)


In [18]:
print("Standard Resnet50")
eval_ddn(STD50, model_paths['std'],x,y,mean,std)

Standard Resnet50
Loading model
Running DDN attack
*****DDN-1000*****
Duration 424.1s 
Success: 100.00% 
Mean L2: 0.0984 
Median L2: 0.0935


In [19]:
print("No Pool Resnet50")
eval_ddn(NoPool50, model_paths['nopool'],x,y,mean,std)

No Pool Resnet50
Loading model
Running DDN attack
*****DDN-1000*****
Duration 417.8s 
Success: 100.00% 
Mean L2: 0.1049 
Median L2: 0.1017


In [20]:
print("Add Pool Resnet50")
eval_ddn(AddPool50, model_paths['addpool'],x,y,mean,std)

Add Pool Resnet50
Loading model
Running DDN attack
*****DDN-1000*****
Duration 137.9s 
Success: 100.00% 
Mean L2: 0.1452 
Median L2: 0.1147


In [21]:
print("No Batch Resnet50")
eval_ddn(NoBatch5, model_paths['nobatch'],x,y,mean,std)

No Batch Resnet50
Loading model
Running DDN attack
*****DDN-1000*****
Duration 384.4s 
Success: 100.00% 
Mean L2: 0.0942 
Median L2: 0.0893
